In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os 

## 1.Get all links for zimuku items

In [2]:
## send the request
url = "http://www.zimuku.net/search?q=&t=onlyst"
result = requests.get(url)
print(result.status_code)

200


In [3]:
## get page number
c = result.content
soup = BeautifulSoup(c,"lxml")
pages =  soup.find('div',{'class':'pagination r clearfix'}).find('div')
page_num = pages.findAll('a')[-1].getText()
page_num = int(page_num)
print(page_num)


12993


In [4]:
## get download link from title link 
def get_down_link(url):
    #url = "http://www.zimuku.net/shooter/57265.html"
    try:
        result = requests.get(url)
        if result.status_code != 200:
            return None
        c = result.content
        soup = BeautifulSoup(c,"lxml")
        down_link =  soup.find('li',{'class':'li dlsub'}).find('a').get('href')
        return down_link
    except:
        return None

In [5]:
## download 
def download_zip(url,folder):
    try:
        result = requests.get(url)
        fname = result.headers['Content-Disposition'].split('"')[1]
        res= result.content
        with open( folder + fname, "wb" ) as f :
                f.write( res )
        return True
    except:
        return False

### set up some global variables

In [6]:
## create doc_list to hold all links 
doc_list = {'title':[],'link':[],'status':[]}
page_start = 9762
page_end = 10000                                ## process 3, 6303 to 10000 pages
folder = 'data/process3/data_download/'

# create folder if folder does not exist
if not os.path.exists(folder):
    os.makedirs(folder)        


In [7]:

## loop through each page
for p in range(page_start,page_end):
    print(p-1, 'success')
    try:
        page_url = url + '&p=' + str(p)
        result = requests.get(page_url)
        #print(result.status_code)
        c = result.content
        soup = BeautifulSoup(c,"lxml")
        docs = soup.find('div',{'class':'box clearfix'}).findAll('div',{'class':'persub clearfix'})

        for doc in docs:

            ## get title 
            link_tag = doc.find('h1',{'class':'title'}).find('a')
            title = link_tag.getText().strip(' \t\n\r@').replace('\n','').replace('\r',' ')
            if title == '_':
                continue
            elif "/" in title:
                 title = title.split("/")[0]

            link='http://www.zimuku.net'+link_tag.get('href')

            ## get download link 
            down_link = get_down_link(link)

            doc_list['title'].append(title)
            doc_list['link'].append(down_link)
        
            ## download file 
            try:
                response = download_zip(down_link,folder)
                doc_list['status'].append(response)
            except:
                doc_list['status'].append(False)
    except:
        continue

print(len(doc_list['link']))

9761 success
9762 success
9763 success
9764 success
9765 success
9766 success
9767 success
9768 success
9769 success
9770 success
9771 success
9772 success
9773 success
9774 success
9775 success
9776 success
9777 success
9778 success
9779 success
9780 success
9781 success
9782 success
9783 success
9784 success
9785 success
9786 success
9787 success
9788 success
9789 success
9790 success
9791 success
9792 success
9793 success
9794 success
9795 success
9796 success
9797 success
9798 success
9799 success
9800 success
9801 success
9802 success
9803 success
9804 success
9805 success
9806 success
9807 success
9808 success
9809 success
9810 success
9811 success
9812 success
9813 success
9814 success
9815 success
9816 success
9817 success
9818 success
9819 success
9820 success
9821 success
9822 success
9823 success
9824 success
9825 success
9826 success
9827 success
9828 success
9829 success
9830 success
9831 success
9832 success
9833 success
9834 success
9835 success
9836 success
9837 success

In [8]:
## export links file 
df = pd.DataFrame(doc_list)
df.to_csv('zimuku_links_v1.csv')
df.ix[0:5]

,link,status,title
0,http://shooter.zimuku.net/download/NjQ2OTZ8JTV...,True,地海战记
1,http://shooter.zimuku.net/download/NjQ2OTR8VHJ...,True,Transformers20th
2,http://shooter.zimuku.net/download/NjQ3MzJ8MDc...,True,07.13.07.Factory.Girl.2006.DVDRip.XViD.AC3.iNT...
3,http://shooter.zimuku.net/download/NjQ2OTN8VHJ...,True,Transformers20th
4,http://shooter.zimuku.net/download/NjYzNjh8QkJ...,True,Amazon Abyss
5,http://shooter.zimuku.net/download/NjQ2OTJ8QnV...,True,burn.notice S01E03


## Now start downloading